In [2]:
import pandas as pd
import os

# 📌 Cài đặt hiển thị
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', None)

# 📂 Đường dẫn chứa các file
RAM_PATH = r"E:\DATA-PROCESSING\Du_an_tot_nghiep\raw_data\raw_data"

# 📥 Hàm đọc và gắn tên hãng
def load_and_prepare(files, airline_name=None):
    dfs = []
    for file in files:
        full_path = os.path.join(RAM_PATH, file)
        print(f"📄 Đang đọc file: {full_path}")
        df = pd.read_csv(full_path)
        df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

        # Thêm tên hãng nếu thiếu
        if "airline" not in df.columns and airline_name:
            df["airline"] = airline_name

        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# 🛫 Load từng hãng
df_vietjet = load_and_prepare(["vietjet_1.csv", "vietjet_2.csv"], airline_name="Vietjet Air")
df_vnairlines = load_and_prepare(["VNairlines_1_1.csv", "VNairlines_2_2.csv"], airline_name="Vietnam Airlines")
df_bamboo = load_and_prepare(["BambooAirway.csv"], airline_name="BAMBOO AIRWAYS")

# 🔗 Gộp toàn bộ dữ liệu
df_all = pd.concat([df_vietjet, df_vnairlines, df_bamboo], ignore_index=True)

# 📊 Kiểm tra hãng bay
print("✅ Hãng bay có trong dữ liệu:", df_all["airline"].unique())

# ✂️ Xóa các cột không cần thiết nếu có
cols_to_drop = ['source.name', 'departure_time', 'arrival_time', 'status']
df_all.drop(columns=[col for col in cols_to_drop if col in df_all.columns], inplace=True)

# 📆 Chuyển đổi cột ngày
df_all['flight_date'] = pd.to_datetime(df_all['date'], errors='coerce')

# ✈️ Tách flight_code và flight_number đúng cách
def split_flight_id(fid):
    try:
        parts = fid.split("-")
        flight_code = parts[1]  # Ví dụ: "VJ568"
        flight_number = ''.join(filter(str.isdigit, flight_code))  # Lấy số: "568"
        return pd.Series([flight_code, flight_number])
    except:
        return pd.Series([None, None])

df_all[['flight_code', 'flight_number']] = df_all['flight_id'].apply(split_flight_id)

# 📆 Tách ngày/tháng/năm, ngày trong tuần
df_all['year'] = df_all['flight_date'].dt.year
df_all['month'] = df_all['flight_date'].dt.month
df_all['day'] = df_all['flight_date'].dt.day
df_all['dow'] = df_all['flight_date'].dt.dayofweek
df_all['is_weekend'] = df_all['dow'].isin([5, 6]).astype(int)

# 🏙️ Map mã sân bay -> tên thành phố
airport_city_map = {
    "SGN": "Ho Chi Minh City", "HAN": "Hanoi", "DAD": "Da Nang", "CXR": "Nha Trang",
    "HUI": "Hue", "DLI": "Da Lat", "PQC": "Phu Quoc", "VII": "Vinh", "VCA": "Can Tho",
    "PXU": "Pleiku", "BMV": "Buon Ma Thuot", "THD": "Thanh Hoa", "VDH": "Dong Hoi",
    "VCL": "Chu Lai", "UIH": "Quy Nhon", "DIN": "Dien Bien", "VKG": "Rach Gia",
    "CAH": "Ca Mau", "VCS": "Con Dao", "BMQ": "Phan Thiet", "CJM": "Con Dao - alt",
    "GLH": "Gia Lai", "NHA": "Nha Trang (alt)", "HAG": "Ha Giang", "YBM": "Yen Bai",
    "HPH": "Hai Phong"
}

df_all['city'] = df_all['origin'].map(airport_city_map)

# 💵 Làm sạch cột giá vé
df_all['price_vnd'] = pd.to_numeric(df_all['price_vnd'], errors='coerce')
df_all = df_all.dropna(subset=['price_vnd'])
df_all['price_vnd'] = df_all['price_vnd'].round(0)

# 💾 Xuất file kết quả
output_path = r"E:\DATA-PROCESSING\Du_an_tot_nghiep\airline_cleaned_clean.csv"
df_all.to_csv(output_path, index=False, encoding="utf-8")
print(f"✅ Đã lưu file sạch: {output_path}")

# 📈 Thống kê số lượng chuyến bay theo hãng
print("\n📊 Số lượng chuyến bay theo hãng:")
print(df_all["airline"].value_counts())


📄 Đang đọc file: E:\DATA-PROCESSING\Du_an_tot_nghiep\raw_data\raw_data\vietjet_1.csv
📄 Đang đọc file: E:\DATA-PROCESSING\Du_an_tot_nghiep\raw_data\raw_data\vietjet_2.csv
📄 Đang đọc file: E:\DATA-PROCESSING\Du_an_tot_nghiep\raw_data\raw_data\VNairlines_1_1.csv
📄 Đang đọc file: E:\DATA-PROCESSING\Du_an_tot_nghiep\raw_data\raw_data\VNairlines_2_2.csv
📄 Đang đọc file: E:\DATA-PROCESSING\Du_an_tot_nghiep\raw_data\raw_data\BambooAirway.csv
✅ Hãng bay có trong dữ liệu: ['Vietjet Air' 'Vietnam Airlines' 'BAMBOO AIRWAYS']
✅ Đã lưu file sạch: E:\DATA-PROCESSING\Du_an_tot_nghiep\airline_cleaned_clean.csv

📊 Số lượng chuyến bay theo hãng:
airline
Vietnam Airlines    1164507
Vietjet Air         1067108
BAMBOO AIRWAYS        23790
Name: count, dtype: int64
